## Group Chat RAG on Tender Documents

#### Importing autogen

In [ ]:
import chromadb
import autogen
import os

#### Initializing 

In [ ]:
config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    file_location=".",
    filter_dict={
        "model": [os.getenv('MODEL_NAME')],
    },
)

print("Models found:", len(config_list))

#### Construct agent

In [ ]:
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
from autogen import AssistantAgent
import chromadb

llm_config = {
    "timeout": 60,
    "cache_seed": 42,
    "config_list": config_list,
    "temperature": 0,
}

# autogen.ChatCompletion.start_logging()
termination_msg = lambda x: isinstance(x, dict) and "TERMINATE" == str(x.get("content", ""))[-9:].upper()

boss = autogen.UserProxyAgent(
    name="Boss",
    is_termination_msg=termination_msg,
    human_input_mode="NEVER",
    system_message="The boss who ask questions and give tasks.",
    code_execution_config=False,  # we don't want to execute code in this case.
    default_auto_reply="Reply `TERMINATE` if the task is done.",
)

boss_aid = RetrieveUserProxyAgent(
    name="Boss_Assistant",
    is_termination_msg=termination_msg,
    system_message="Assistant who has extra content retrieval power for solving difficult problems.",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=3,
    retrieve_config={
        "task": "qa",
		"docs_path": [
			"./resources/cleaned/AUTONOMOUS ROBOT Evaluation Report (No Award) 170122.md",
			"./resources/cleaned/Development of Financial Declaration App_SOW (Final).md",
			"./resources/cleaned/Development of Mobile App- DC Inno Evaluation report V3_15Feb.md"
		],
        # "docs_path": "https://raw.githubusercontent.com/microsoft/FLAML/main/website/docs/Examples/Integrate%20-%20Spark.md",
        "chunk_token_size": 1000,
        "model": config_list[0]["model"],
        "client": chromadb.PersistentClient(path="/tmp/chromadb_brand_new_l"),
        "collection_name": "groupchat",
        "get_or_create": True,
    },
    code_execution_config=False,  # we don't want to execute code in this case.
)

admin = AssistantAgent(
    name="Senior_Admin_Officer",
    is_termination_msg=termination_msg,
    system_message="You are a senior administration officer. For every sentence you write, try to cite the <Excerpt Citation: id> as [id] at the end of each sentence. Reply `TERMINATE` in the end when everything is done.",
    llm_config=llm_config,
)

reviewer = AssistantAgent(
    name="Quality_Reviewer",
    is_termination_msg=termination_msg,
    system_message="You are a quality reviewer. Reply `TERMINATE` in the end when everything is done.",
    llm_config=llm_config,
)

def _reset_agents():
    boss.reset()
    boss_aid.reset()
    admin.reset()
    reviewer.reset()


def rag_chat(query):
    _reset_agents()
    groupchat = autogen.GroupChat(
        agents=[boss_aid, admin, reviewer], messages=[], max_round=12, speaker_selection_method="round_robin"
    )

    manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

    # Start chatting with boss_aid as this is the user proxy agent.
    boss_aid.initiate_chat(
        manager,
        problem=query,
        n_results=3,
    )

In [ ]:
rag_chat("What is the negotiated target price of the mobile app?")